## Catalog Combiner
This notebook will combine three seperate filter catalogs into a single final catalog

In [1]:
#standard imports
%matplotlib inline
import seWrapper
from imp import reload
import os
import matplotlib.pyplot as plt
import numpy as np
reload(seWrapper)

import pandas as pd

#### Replace the names of the files below with the final catalogs produced by the zero point step. 

In [7]:
g_band_data = pd.read_csv('n4486_g_final.csv')
r_band_data = pd.read_csv('n4486_r_final.csv')
i_band_data = pd.read_csv('n4486_i_final.csv')

#### Create catalog objects to store the photometry.
These are custom objects that can handle detection/non-detection conflicts across filters. We need to create one for each filter. The cells below should do this correctly.

In [ ]:
se_g = seWrapper.seCatalog()
se_g.create_new_catalog_arrs(g_band_data['ra'],\
                        g_band_data['dec'],\
                           fil='g',\
                        mags = g_band_data['mag'],
                        fwhm = g_band_data['fwhm'],
                       flags = g_band_data['flag'],
                       ellipticity = g_band_data['ellip'],
                        merr = g_band_data['merr'])

In [9]:
se_r = seWrapper.seCatalog()
se_r.create_new_catalog_arrs(r_band_data['ra'],\
                        r_band_data['dec'],\
                           fil='r',\
                        mags = r_band_data['mag'],
                        fwhm = r_band_data['fwhm'],
                       flags = r_band_data['flag'],
                       ellipticity = r_band_data['ellip'],
                            merr = r_band_data['merr'])

In [10]:
se_i = seWrapper.seCatalog()
se_i.create_new_catalog_arrs(i_band_data['ra'],\
                        i_band_data['dec'],\
                           fil='i',\
                        mags = i_band_data['mag'],
                        fwhm = i_band_data['fwhm'],
                       flags = i_band_data['flag'],
                       ellipticity = i_band_data['ellip'],
                            merr = i_band_data['merr'])

#### Next merge the catalogs together into one master catalog containg all three filters.
Run the two steps below to merge the se_r and se_i catalogs into the se_g catalog. Note that the steps below can take a while to run.

In [11]:
se_g.mergeCatalog(se_r)


In [12]:
se_g.mergeCatalog(se_i)

#### Only select objects detected in all filters.
For purposes of creating the master photometric catalog, we are only interested in objects that are detected in all three filters. Non-detections in any filter are stored as 99.9999 magnitude, so the line below will only select objects with photometry in all three filters.

In [13]:
all_detections = (se_g.mags['g'] < 90) & (se_g.mags['r'] < 90) & (se_g.mags['i'] < 90) 

#### Place all catalogs into a pandas dataframe for easy writing and manipulation.
Run the two cells below to store the final catalog in an easy to work with format.

In [16]:
#Now that catalogs are joined, put everything in a pandas data frame. Note that if I had written
#my catalog code a year later, it would be written with the intention of doing this from the beginning...
catalog_dict = {}

#get the ra and dec from the coordinate object and place them in a pandas series
catalog_dict['ra'] = pd.Series(se_g.coords.ra.degree)
catalog_dict['dec'] = pd.Series(se_g.coords.dec.degree)

#put the mags in pandas series:
catalog_dict['g_mag'] = pd.Series(se_g.mags['g'])
catalog_dict['r_mag'] = pd.Series(se_g.mags['r'])
catalog_dict['i_mag'] = pd.Series(se_g.mags['i'])

#same for the merrs:
catalog_dict['g_merr'] = pd.Series(se_g.merr['g'])
catalog_dict['r_merr'] = pd.Series(se_g.merr['r'])
catalog_dict['i_merr'] = pd.Series(se_g.merr['i'])

#same for the fwhm:
catalog_dict['g_fwhm'] = pd.Series(se_g.fwhm['g'])
catalog_dict['r_fwhm'] = pd.Series(se_g.fwhm['r'])
catalog_dict['i_fwhm'] = pd.Series(se_g.fwhm['i'])

#same for the ellipticity:
catalog_dict['g_ellip'] = pd.Series(se_g.ellipticity['g'])
catalog_dict['r_ellip'] = pd.Series(se_g.ellipticity['r'])
catalog_dict['i_ellip'] = pd.Series(se_g.ellipticity['i'])

#same for the flags:
catalog_dict['g_flags'] = pd.Series(se_g.flags['g'])
catalog_dict['r_flags'] = pd.Series(se_g.flags['r'])
catalog_dict['i_flags'] = pd.Series(se_g.flags['i'])

In [17]:
catalog = pd.DataFrame(catalog_dict)
#only keep objects detected in all filters.
catalog = catalog.ix[all_detections,:]

#### Finally, write out the catalog.
Write the pandas dataframe to a .csv format catalog. Be sure to re-name the catalog to an appropriate designation. After this, the catalog should be easy to read in using pandas and the columns should be straightforward to manipulate. 

In [18]:
catalog.to_csv('n4486_phot_combined.csv')